In [0]:
%load_ext autoreload
%autoreload 2

## DimUser

In [0]:
from pyspark.sql import functions as F, types as T

#### Autoloader

In [0]:
bronze_path = "abfss://bronze@storageagspotifyazure.dfs.core.windows.net/"
silver_path = "abfss://silver@storageagspotifyazure.dfs.core.windows.net/"

In [0]:
df_user = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option(
        "cloudFiles.schemaLocation",
        f"{silver_path}/DimUser/checkpoint",
    )
    .load(f"{bronze_path}/DimUser")
)

In [0]:
df_user = df_user.withColumn("user_name", F.upper(F.col("user_name")))
# display(df_user, checkpointLocation=f"{silver_path}/DimUser/checkpoint", outputMode="append")

In [0]:
%run /Workspace/Users/ahampersonal18113@gmail.com/agspotifyazure_dab/utils/transformations.py

In [0]:
df_user_obj = resusable()

df_user = df_user_obj.dropColumns(df_user, ["_rescued_data"])
df_user = df_user.dropDuplicates(["user_id"])
# display(df_user, checkpointLocation=f"{silver_path}/DimUser/checkpoint", outputMode="append")

In [0]:
dbutils.fs.rm(
    f"{silver_path}/DimUser/checkpoint", recurse=True
)  # clean the number of states

In [0]:
df_user.writeStream.format("delta").option(
    "checkpointLocation",
    f"{silver_path}/DimUser/checkpoint",
).option("mergeSchema", "true").outputMode("append").trigger(availableNow=True).option(
    "path", f"{silver_path}/DimUser/data"
).toTable(
    "spotify_cata.silver.DimUser"
)

## DimArtist

In [0]:
df_artist = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", f"{silver_path}/DimArtist/checkpoint")
    .load(f"{bronze_path}/DimArtist")
)

In [0]:
# display(df_artist, checkpointLocation=f"{silver_path}/DimArtist/checkpoint", outputMode="append")

In [0]:
df_artist_obj = resusable()
df_artist = df_artist_obj.dropColumns(df_artist, ["_rescued_data"])
df_artist = df_artist.dropDuplicates(["artist_id"])
# display(df_artist, checkpointLocation=f"{silver_path}/DimArtist/checkpoint", outputMode="append")

In [0]:
dbutils.fs.rm(f"{silver_path}/DimArtist/checkpoint", recurse=True)

In [0]:
df_artist.writeStream.format("delta").option(
    "checkpointLocation",
    f"{silver_path}/DimArtist/checkpoint",
).option("mergeSchema", "true").outputMode("append").trigger(availableNow=True).option(
    "path", f"{silver_path}/DimArtist/data"
).toTable(
    "spotify_cata.silver.DimArtist"
)

## DimTrack

In [0]:
df_track = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", f"{silver_path}/DimTrack/checkpoint")
    .load(f"{bronze_path}/DimTrack")
)

In [0]:
# display(df_track, checkpointLocation=f"{silver_path}/DimTrack/checkpoint", outputMode="append")

In [0]:
df_track = df_track.withColumn(
    "durationFlag",
    F.when(F.col("duration_sec") < 150, "Low")
    .when(F.col("duration_sec") > 300, "High")
    .otherwise("Medium"),
)

df_track = df_track.withColumn(
    "track_name", F.regexp_replace(F.col("track_name"), "-", "")
)

df_track_obj = resusable()
df_track = df_track_obj.dropColumns(df_track, ["_rescued_data"])

# display(df_track, checkpointLocation=f"{silver_path}/DimTrack/checkpoint", outputMode="append")

In [0]:
dbutils.fs.rm(f"{silver_path}/DimTrack/checkpoint", recurse=True)

In [0]:
df_track.writeStream.format("delta").option(
    "checkpointLocation",
    f"{silver_path}/DimTrack/checkpoint",
).option("mergeSchema", "true").outputMode("append").trigger(availableNow=True).option(
    "path", f"{silver_path}/DimTrack/data"
).toTable("spotify_cata.silver.DimTrack")

## DimDate

In [0]:
df_date = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", f"{silver_path}/DimDate/checkpoint")
    .load(f"{bronze_path}/DimDate")
)

In [0]:
df_date = resusable().dropColumns(df_date, ["_rescued_data"])

In [0]:
dbutils.fs.rm(f"{silver_path}/DimDate/checkpoint", recurse=True)

In [0]:
df_date.writeStream.format("delta").option(
    "checkpointLocation",
    f"{silver_path}/DimDate/checkpoint",
).option("mergeSchema", "true").outputMode("append").trigger(availableNow=True).option(
    "path", f"{silver_path}/DimDate/data"
).toTable("spotify_cata.silver.DimDate")

## FactStream

In [0]:
df_fact = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", f"{silver_path}/FactStream/checkpoint")
    .load(f"{bronze_path}/FactStream")
)

In [0]:
df_fact = resusable().dropColumns(df_fact, ["_rescued_data"])

In [0]:
dbutils.fs.rm(f"{silver_path}/FactStream/checkpoint", recurse=True)

In [0]:
df_fact.writeStream.format("delta").option(
    "checkpointLocation",
    f"{silver_path}/FactStream/checkpoint",
).option("mergeSchema", "true").outputMode("append").trigger(availableNow=True).option(
    "path", f"{silver_path}/FactStream/data"
).toTable(
    "spotify_cata.silver.FactStream"
)

In [0]:
%sql
select * from spotify_cata.gold.dimtrack
-- where __END_AT is not null
where track_id in (46, 5)